# Data Story

1. Questions
    - Comparisons
    - Counts
    - Trends
    - Plots (Bar/Histogram/Scatter/Time-Series)
    - Cross-tabs
2. Insights
    - Correlations
    - Hypotheses
3. Narrative
    - Present in a story form, needs to flow
    - What trends/relationships would make it more complete?

**Notes:**

- Date/Time needs to be categorized before being used in ML models
- Only choose a few columns to explore in detail, can always go back and explore further after starting ML
- *Histogram of diff in appointment-date relative to scheduled-date*

## Import clean DataFrame from 2.0-jkg-data-wrangling

In [ ]:
# Package imports
import pandas as pd

# import pickle file
df_clean = pd.read_pickle('../data/interim/clean_df.pickle')


In [ ]:
# Appointments DataFrame
df_appointments = df_clean.copy()

# Patients DataFrame
df_patients = df_clean.copy()

## Visualization TODOs

- [x] Create DataFrame of Patients w/ duplicates removed (this would also help identify outliers, patients w/tons of appointments)
- Histogram of DateDiff (overall and missed appointments)
- Crosstab of SMS_sent and noshow

In [ ]:
df_patients.drop(columns=['Scheduled_Date', 'Appointment_Date', 'SMS_sent', 'Age'], inplace=True)
df_patients.No_show.replace(('Yes', 'No'), (1, 0), inplace=True)

In [ ]:
patient_appointments = df_patients.groupby('Patient_ID')['Appointment_ID'].count()
patient_noshows = df_patients.groupby('Patient_ID')['No_show'].sum()

In [ ]:
df_patients.drop(columns=['Appointment_ID', 'No_show'], inplace=True)
df_patients.drop_duplicates(inplace=True)
df_patients.set_index('Patient_ID', inplace=True)

In [ ]:
patient_appointments.name = 'total_appointments'
df_patients = df_patients.join(patient_appointments)

patient_noshows.name = 'noshow_appointments'
df_patients = df_patients.join(patient_noshows)

df_patients.sort_index(inplace=True)

In [ ]:
# Calculate Date Difference (Appointment - Scheduled)

# (df_clean['Appointment_Date'] - df_clean['Scheduled_Date'])

# need to recast scheduled date as just date since appointment date does not have time included
# look for potential outliers of backdated appointments

df_appointments['Scheduled_Date_Only'] = df_appointments['Scheduled_Date'].dt.date

In [ ]:
df_appointments.head()

In [ ]:
df_appointments.info()

In [ ]:
df_appointments['Scheduled_Date_Only'] = pd.to_datetime(df_appointments['Scheduled_Date_Only'])

In [ ]:
df_appointments['Date_Diff'] = df_appointments['Appointment_Date'] - df_appointments['Scheduled_Date_Only']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#plt.hist(df_appointments['Date_Diff'])

df_appointments['Date_Diff'].astype('timedelta64[D]').hist(bins=100)

plt.show()

In [ ]:
df_appointments['Date_Diff'].astype('timedelta64[D]')

In [ ]:
#df_appointments['Date_Diff'].value_counts().sort_index()

date_diff = df_appointments['Date_Diff'].astype('timedelta64[D]')

date_diff_gt_one = date_diff.loc[date_diff > 0]

In [ ]:
date_diff_gt_one.value_counts().sort_index()

In [ ]:
plt.clf()

date_diff_gt_one.hist(bins=200)

plt.show()

In [ ]:
# create two dataframes, one with no-show=yes and one with no-show=no, count then merge
#df_clean

#df_patients[['Patient_ID', 'No'

In [ ]:
df_patients_no_show = df_patients.loc[df_patients['No_show'] == 'Yes'].groupby('Patient_ID').count()
#df_patients[['Patient_ID', 'No_show']].loc[:, df_patients['No_show'] == True]

df_patients_total_appointments = df_patients.groupby('Patient_ID').count()

df_patients_no_show['no_show_count'] = df_patients_no_show['No_show']

df_patients_total_appointments['total_appointments'] = df_patients_total_appointments['No_show']

df_patients_appointment_counts = df_patients_total_appointments.merge(df_patients_no_show, how='left', left_index=True, right_index=True)

In [ ]:
df_patients_appointment_counts = df_patients_appointment_counts[['no_show_count', 'total_appointments']].fillna(0)

In [ ]:
df_patients_appointment_counts.info()

In [ ]:
df_patients_appointment_counts['no_show_count'] = df_patients_appointment_counts['no_show_count'].astype('int64')

In [ ]:
df_patients_appointment_counts['no_show_pct'] = df_patients_appointment_counts['no_show_count'] / df_patients_appointment_counts['total_appointments']

In [ ]:
df_patients_appointment_counts